In [ ]:
import numpy as np
import pandas as pd 
import plotly.graph_objects as go
import json
import os
import matplotlib.pyplot as plt

## Purpose

The purpose of this file is to analyze, for a fixed set of other params, what the optimal price is under delay differentiation. 

- Analyze what is the optimal price
    - How does using it compare to a baseline of 0 price
        - Total cost 
        - Wait times
        - Anything else interesting 

In [ ]:

#Folder of the grid search you want to analyze, could be just one set of tested params in the 
#grid search 

gridsearch_folder = 'results/gridsearch_May7-06-05-4PM'

result_folders = [name for name in os.listdir(gridsearch_folder)]


warmup_period = 1500 # num customers to skip counting at the beginning 

## Warm Up Time Plots

In [ ]:
def plot_warmup_graphs(gridsearch_folder, param_tokens = []):
    """
    For each set of parameters, plot the number of customers 
    already in each queue or being served (slow/fast) at each arrival time.
    Allows us to determine things go to steady state/when steady state happens.
    Plots are given by time and customer arrival index. 
    Do this for only one run, just to show that we are approx. evaluating the results at steady state. 
    Can use this plot to determine the number of warmup customers - how many initial records we discard
    (if any) because they are not the steadystate behavior. 
    
    gridsearch_folder - the folder of parameter sets to plot 
    param_tokens = list of params to label the runs by (parameter changes which defined the runs)
    """
    result_folders = [name for name in os.listdir(gridsearch_folder)]
    
    result_folders.sort()
    
    for param_results in result_folders:
     
        
        with open(f'{gridsearch_folder}/{param_results}/params.json') as file:
            param_dict = json.load(file)
            
            
        param_string = []
        for token in param_tokens:
            param_string.append(f'{token}: {param_dict[token]}')
        param_string = ', '.join(param_string)
        
        run_file = f'{gridsearch_folder}/{param_results}/customer_results_run_{0}.csv'
        run_df = pd.read_csv(run_file, index_col=0)
        
        for charge_type in ['Fast', 'Slow']:

            x_values = list(run_df['arrival_time'])
            y_values = list(run_df[f'L_{charge_type.lower()}'])

            # Create the plot
            fig = go.Figure()
            
            fig.update_yaxes(rangemode="tozero")

            # Add a line trace
            fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='Line'))

            # Update layout
            fig.update_layout(title=f'Number of Customers in {charge_type} Queue Over Time ({param_string})',
                              xaxis_title='Time (Minutes)',
                              yaxis_title=f'Number of Customers in {charge_type} Queue')

            # Show the plot
            fig.show()

#         for charge_type in ['Fast', 'Slow']:

#             x_values = list(run_df.index)
#             y_values = list(run_df[f'L_{charge_type.lower()}'])

#             # Create the plot
#             fig = go.Figure()
#             fig.update_yaxes(rangemode="tozero")
#             # Add a line trace
#             fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='Line'))

#             # Update layout
#             fig.update_layout(title=f'Number of Customers in {charge_type} Queue by Arrival ({param_string})',
#                               xaxis_title='Customer Arrival Index',
#                               yaxis_title=f'Number of Customers in {charge_type} Queue')

#             # Show the plot
#             fig.show()

In [ ]:
plot_warmup_graphs(gridsearch_folder, ['p_fast'])

## Summary Tables 

Gives summary table for each set of parameters. Can handle summary tables where multiple parameters are being changed (input the names of all changed params in relevant_params). 

All values in these tables are per-capita in the simulation for that type of customer.

In [ ]:
def make_summary_tables(gridsearch_folder, relevant_params, n_warmup_customers = 0):
    
    """
    gridsearch_folder : location of results
    relevant_params: strings (keys of the data) of the params that were changed/are relevant to the summary
    n_warmup_customers: number of initial rows to drop bc of warmup in the summary 
    
    """
    
    result_folders = [name for name in os.listdir(gridsearch_folder)]
    
    all_customers = pd.DataFrame(columns=relevant_params +['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price', 'cost_from_wait'])
    fast_customers = pd.DataFrame(columns=relevant_params +['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price', 'cost_from_wait'])
    slow_customers = pd.DataFrame(columns=relevant_params +['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price', 'cost_from_wait'])
    ds_customers = pd.DataFrame(columns=relevant_params +['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price', 'cost_from_wait'])
    ps_customers =  pd.DataFrame(columns=relevant_params +['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price', 'cost_from_wait'])
    
    total_all_customers = pd.DataFrame(columns=relevant_params +['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price', 'cost_from_wait'])
    total_fast_customers = pd.DataFrame(columns=relevant_params +['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price', 'cost_from_wait'])
    total_slow_customers = pd.DataFrame(columns=relevant_params +['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price', 'cost_from_wait'])
    total_ds_customers = pd.DataFrame(columns=relevant_params +['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price', 'cost_from_wait'])
    total_ps_customers =  pd.DataFrame(columns=relevant_params +['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price', 'cost_from_wait'])
    
    for param_results in result_folders:
        
        avg_all_customers = pd.DataFrame(columns=['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price'])
        avg_fast_customers = pd.DataFrame(columns=['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price'])
        avg_slow_customers = pd.DataFrame(columns=['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price'])
        avg_ds_customers = pd.DataFrame(columns=['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price'])
        avg_ps_customers =  pd.DataFrame(columns=['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price'])
        
        sum_all_customers = pd.DataFrame(columns=['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price'])
        sum_fast_customers = pd.DataFrame(columns=['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price'])
        sum_slow_customers = pd.DataFrame(columns=['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price'])
        sum_ds_customers = pd.DataFrame(columns=['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price'])
        sum_ps_customers =  pd.DataFrame(columns=['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price'])
        

        with open(f'{gridsearch_folder}/{param_results}/params.json') as file:
            param_dict = json.load(file)
            
        num_runs = param_dict['num_runs']
        #num_customers = param_dict['num_customers'] - n_warmup_customers
        
        for run in range(num_runs):

            
            run_file = f'{gridsearch_folder}/{param_results}/customer_results_run_{run}.csv'
            runs_df=pd.read_csv(run_file, skiprows = range(1, n_warmup_customers+1), index_col = 0)

            
            avg_cols = runs_df[['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price']]
            sum_cols = avg_cols.sum() #total
            avg_cols=avg_cols.mean() #results on avg for this run per customer
            avg_all_customers.loc[len(avg_all_customers)]=avg_cols
            sum_all_customers.loc[len(sum_all_customers)]=sum_cols
            
            
            avg_cols = runs_df[runs_df['charger_choice'] == 'fast'][['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price']]
            sum_cols = avg_cols.sum()
            avg_cols=avg_cols.mean() #results on avg for this run per customer
            avg_fast_customers.loc[len(avg_fast_customers)]=avg_cols 
            sum_fast_customers.loc[len(sum_fast_customers)]=sum_cols
            
            avg_cols = runs_df[runs_df['charger_choice'] == 'slow'][['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price']]
            sum_cols = avg_cols.sum()
            avg_cols=avg_cols.mean() #results on avg for this run per customer
            avg_slow_customers.loc[len(avg_slow_customers)]=avg_cols
            sum_slow_customers.loc[len(sum_slow_customers)]=sum_cols
            
            avg_cols = runs_df[runs_df['sens_type'] == 'delay'][['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price']]
            sum_cols = avg_cols.sum()
            avg_cols=avg_cols.mean() #results on avg for this run per customer
            avg_ds_customers.loc[len(avg_ds_customers)]=avg_cols
            sum_ds_customers.loc[len(sum_ds_customers)]=sum_cols
            
            avg_cols = runs_df[runs_df['sens_type'] == 'price'][['true_cost','true_total_wait','true_queue_wait','true_servetime','charge_price']]
            sum_cols = avg_cols.sum()
            avg_cols=avg_cols.mean() #results on avg for this run per customer
            avg_ps_customers.loc[len(avg_ps_customers)]=avg_cols
            sum_ps_customers.loc[len(sum_ps_customers)]=sum_cols

        #avg per customer values over all runs
        avg_all_customers = avg_all_customers.mean()
        avg_fast_customers = avg_fast_customers.mean()
        avg_slow_customers = avg_slow_customers.mean()
        avg_ds_customers = avg_ds_customers.mean()
        avg_ps_customers = avg_ps_customers.mean()
        
        #avg total over all runs
        sum_all_customers = sum_all_customers.mean()
        sum_fast_customers = sum_fast_customers.mean()
        sum_slow_customers = sum_slow_customers.mean()
        sum_ds_customers = sum_ds_customers.mean()
        sum_ps_customers = sum_ps_customers.mean()
        
        
        avg_results_dfs = [avg_all_customers, avg_fast_customers, avg_slow_customers, avg_ds_customers, avg_ps_customers]
        sum_results_dfs = [sum_all_customers, sum_fast_customers, sum_slow_customers, sum_ds_customers, sum_ps_customers]
        
            
        for i in range(len(avg_results_dfs)):
                
            avg_results_dfs[i]['cost_from_wait'] = avg_results_dfs[i]['true_cost']-avg_results_dfs[i]['charge_price']
            sum_results_dfs[i]['cost_from_wait'] = sum_results_dfs[i]['true_cost']-sum_results_dfs[i]['charge_price']
            for param_token in relevant_params:
                avg_results_dfs[i][param_token] = param_dict[param_token]
                sum_results_dfs[i][param_token] = param_dict[param_token]
            
    
        all_customers.loc[len(all_customers.index)]= avg_all_customers
        fast_customers.loc[len(fast_customers.index)]= avg_fast_customers
        slow_customers.loc[len(slow_customers.index)]= avg_slow_customers
        ds_customers.loc[len(ds_customers.index)]= avg_ds_customers
        ps_customers.loc[len(ps_customers.index)]= avg_ps_customers
        
        total_all_customers.loc[len(total_all_customers.index)]= sum_all_customers
        total_fast_customers.loc[len(total_fast_customers.index)]= sum_fast_customers
        total_slow_customers.loc[len(total_slow_customers.index)]= sum_slow_customers
        total_ds_customers.loc[len(total_ds_customers.index)]= sum_ds_customers
        total_ps_customers.loc[len(total_ps_customers.index)]= sum_ps_customers
        
    total_all_customers = total_all_customers.sort_values(relevant_params)
    total_fast_customers = total_fast_customers.sort_values(relevant_params)
    total_slow_customers = total_slow_customers.sort_values(relevant_params)
    total_ds_customers = total_ds_customers.sort_values(relevant_params)
    total_ps_customers = total_ps_customers.sort_values(relevant_params)
        
    print("Per Capita: All Customers")
    
    all_customers = all_customers.sort_values(relevant_params)
    
    display(all_customers)
    
    print("Per Capita: Fast Charging Customers")
    
    fast_customers = fast_customers.sort_values(relevant_params)
    
    display(fast_customers)
    
    print("Per Capita: Slow Charging Customers")
    slow_customers = slow_customers.sort_values(relevant_params)
    
    display(slow_customers)
    
    print("Per Capita: Delay Sensitive Customers")
    ds_customers = ds_customers.sort_values(relevant_params)
    
    display(ds_customers)
    
    print("Per Capita: Price Sensitive Customers")
    
    ps_customers = ps_customers.sort_values(relevant_params)
    
    display(ps_customers)
    
    
    return all_customers, total_all_customers,\
            fast_customers, total_fast_customers,\
            slow_customers, total_slow_customers,\
            ds_customers, total_ds_customers,\
            ps_customers, total_ps_customers
    

In [ ]:
#First df is per-capita tables for that type of customer
#Second df is total tables for that type of customer

all_customers, total_all_customers,\
fast_customers,total_fast_customers,\
slow_customers, total_slow_customers,\
ds_customers, total_ds_customers,\
ps_customers, total_ps_customers = make_summary_tables(gridsearch_folder, ['p_fast'], 1500)

## Comparing the Impact of Price (on Consistent Non-Price Params)

We explore:

- How does cost change with p_fast? How much of this cost is from each group?
- How does average wait time (per customer) change with p_fast? Broken down by group?


Plots
   - Price vs. total cost 
   - Price vs. cost, per customer
   
   - Price vs. total cost for fast/slow/all customers 
   - Price vs. cost per fast/slow/all customers 
   
   - Price vs. total cost for ds/ps/all customers
   - Price vs. cost per ds/ps/all customers
   
   - Price vs. total wait
   - Price vs. wait, per customer
   
   - Price vs. total wait for fast/slow/all customers 
   - Price vs. wait per fast/slow/all customers 
   
   - Price vs. total wait for ds/ps/all customers
   - Price vs. wait per ds/ps/all customers

In [ ]:
#Total cost for all customers - plot

fig = go.Figure()
fig.update_yaxes(rangemode="tozero")

total_cost = list(total_all_customers['true_cost'])
p_fasts = list(total_all_customers['p_fast'])

min_tc_idx = np.argmin(total_cost) 


fig.add_trace(go.Scatter(x=p_fasts, y=total_cost, mode='lines', name='Total Cost'))

fig.add_trace(go.Scatter(x=[p_fasts[min_tc_idx]], y=[total_cost[min_tc_idx]],mode='markers', marker=dict(color='blue', size=10), name=f'Minimizing Price: {p_fasts[min_tc_idx]}'))

fig.update_layout(title='Effect of Fast Charger Price on Total Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Total Cost (Delay and Price)')

fig.show()

#Per capita cost for any customer - plot

fig = go.Figure()
fig.update_yaxes(rangemode="tozero")

total_cost = list(all_customers['true_cost'])
p_fasts = list(all_customers['p_fast'])

min_tc_idx = np.argmin(total_cost) 


fig.add_trace(go.Scatter(x=p_fasts, y=total_cost, mode='lines', name='Cost Per Capita'))

fig.add_trace(go.Scatter(x=[p_fasts[min_tc_idx]], y=[total_cost[min_tc_idx]],mode='markers', marker=dict(color='blue', size=10), name=f'Minimizing Price: {p_fasts[min_tc_idx]}'))

fig.update_layout(title='Effect of Fast Charger Price on Cost Per Capita',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost Per Capita (Delay and Price)')

fig.show()

In [ ]:
def plot_price_impact(y_variable, title_str, all_customers, total_all_customers, \
                      fast_customers, total_fast_customers,\
                      slow_customers, total_slow_customers,\
                     ds_customers, total_ds_customers,\
                     ps_customers, total_ps_customers):
    
    
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    total_cost = list(total_all_customers[y_variable])
    p_fasts = list(total_all_customers['p_fast'])

    total_cost_fast = list(total_fast_customers[y_variable])
    total_cost_slow = list(total_slow_customers[y_variable])

    min_tc_idx = np.argmin(total_cost) 

    min_fast_idx = np.argmin(total_cost_fast)

    min_slow_idx = np.argmin(total_cost_slow)

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost, mode='lines', line =dict(color='blue'), name=f'Total {title_str}'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_tc_idx]], y=[total_cost[min_tc_idx]],mode='markers', marker=dict(color='blue', size=10), name=f'Optimal Price: {p_fasts[min_tc_idx]}'))

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost_fast, mode='lines', line =dict(color='red'), name=f'Total {title_str} (Fast)'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_fast_idx]], y=[total_cost_fast[min_fast_idx]],mode='markers', marker=dict(color='red', size=10), name=f'Optimal Price (Fast): {p_fasts[min_fast_idx]}'))

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost_slow, mode='lines', line =dict(color='green'), name=f'Total {title_str} (Slow)'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_slow_idx]], y=[total_cost_slow[min_slow_idx]],mode='markers', marker=dict(color='green', size=10), name=f'Optimal Price (Slow): {p_fasts[min_slow_idx]}'))


    fig.update_layout(title=f'Effect of Fast Charger Price on Total {title_str}',
                              xaxis_title='Fast Charger Price',
                              yaxis_title=f'Total {title_str}')

    fig.show()

    #Per capita cost for customers who enter the slow vs. fast queue - plot

    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    total_cost = list(all_customers[y_variable])
    p_fasts = list(all_customers['p_fast'])

    total_cost_fast = list(fast_customers[y_variable])
    total_cost_slow = list(slow_customers[y_variable])

    min_tc_idx = np.argmin(total_cost) 

    min_fast_idx = np.argmin(total_cost_fast)

    min_slow_idx = np.argmin(total_cost_slow)

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost, mode='lines', line =dict(color='blue'), name=f'{title_str} Per Capita'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_tc_idx]], y=[total_cost[min_tc_idx]],mode='markers', marker=dict(color='blue', size=10), name=f'Optimal Price: {p_fasts[min_tc_idx]}'))

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost_fast, mode='lines', line =dict(color='red'), name=f'{title_str} Per Capita (Fast)'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_fast_idx]], y=[total_cost_fast[min_fast_idx]],mode='markers', marker=dict(color='red', size=10), name=f'Optimal Price (Fast): {p_fasts[min_fast_idx]}'))

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost_slow, mode='lines', line =dict(color='green'), name=f'{title_str} Per Capita (Slow)'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_slow_idx]], y=[total_cost_slow[min_slow_idx]],mode='markers', marker=dict(color='green', size=10), name=f'Optimal Price (Slow): {p_fasts[min_slow_idx]}'))


    fig.update_layout(title=f'Effect of Fast Charger Price on {title_str} Per Capita',
                              xaxis_title='Fast Charger Price',
                              yaxis_title=f'{title_str} Per Capita')

    fig.show()


    #DS/PS Customers

    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    total_cost = list(total_all_customers[y_variable])
    p_fasts = list(total_all_customers['p_fast'])

    total_cost_fast = list(total_ds_customers[y_variable])
    total_cost_slow = list(total_ps_customers[y_variable])

    min_tc_idx = np.argmin(total_cost) 

    min_fast_idx = np.argmin(total_cost_fast)

    min_slow_idx = np.argmin(total_cost_slow)

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost, mode='lines', line =dict(color='blue'), name=f'Total {title_str}'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_tc_idx]], y=[total_cost[min_tc_idx]],mode='markers', marker=dict(color='blue', size=10), name=f'Optimal Price: {p_fasts[min_tc_idx]}'))

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost_fast, mode='lines', line =dict(color='purple'), name=f'Total {title_str} (Delay Sensitive)'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_fast_idx]], y=[total_cost_fast[min_fast_idx]],mode='markers', marker=dict(color='purple', size=10), name=f'Optimal Price (DS): {p_fasts[min_fast_idx]}'))

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost_slow, mode='lines', line =dict(color='orange'), name=f'Total {title_str} (Price Sensitive)'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_slow_idx]], y=[total_cost_slow[min_slow_idx]],mode='markers', marker=dict(color='orange', size=10), name=f'Optimal Price (PS): {p_fasts[min_slow_idx]}'))


    fig.update_layout(title=f'Effect of Fast Charger Price on Total {title_str}',
                              xaxis_title='Fast Charger Price',
                              yaxis_title=f'Total {title_str}')

    fig.show()


    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    total_cost = list(all_customers[y_variable])
    p_fasts = list(all_customers['p_fast'])

    total_cost_fast = list(ds_customers[y_variable])
    total_cost_slow = list(ps_customers[y_variable])

    min_tc_idx = np.argmin(total_cost) 

    min_fast_idx = np.argmin(total_cost_fast)

    min_slow_idx = np.argmin(total_cost_slow)

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost, mode='lines', line =dict(color='blue'), name=f'{title_str} Per Capita'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_tc_idx]], y=[total_cost[min_tc_idx]],mode='markers', marker=dict(color='blue', size=10), name=f'Optimal Price: {p_fasts[min_tc_idx]}'))

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost_fast, mode='lines', line =dict(color='purple'), name=f'{title_str} Per Capita (Delay Sensitive)'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_fast_idx]], y=[total_cost_fast[min_fast_idx]],mode='markers', marker=dict(color='purple', size=10), name=f'Optimal Price (DS): {p_fasts[min_fast_idx]}'))

    fig.add_trace(go.Scatter(x=p_fasts, y=total_cost_slow, mode='lines', line =dict(color='orange'), name=f'{title_str} Per Capita (Price Sensitive)'))

    fig.add_trace(go.Scatter(x=[p_fasts[min_slow_idx]], y=[total_cost_slow[min_slow_idx]],mode='markers', marker=dict(color='orange', size=10), name=f'Optimal Price (PS): {p_fasts[min_slow_idx]}'))


    fig.update_layout(title=f'Effect of Fast Charger Price on {title_str} Per Capita',
                              xaxis_title='Fast Charger Price',
                              yaxis_title=f'{title_str} Per Capita')

    fig.show()

In [ ]:
plot_price_impact('true_cost', "Cost", all_customers, total_all_customers, \
                      fast_customers, total_fast_customers,\
                      slow_customers, total_slow_customers,\
                     ds_customers, total_ds_customers,\
                     ps_customers, total_ps_customers)

In [ ]:
plot_price_impact('true_total_wait', 'System Wait Time', all_customers, total_all_customers, \
                      fast_customers, total_fast_customers,\
                      slow_customers, total_slow_customers,\
                     ds_customers, total_ds_customers,\
                     ps_customers, total_ps_customers)

In [ ]:
plot_price_impact('true_queue_wait', 'Queue Wait Time', all_customers, total_all_customers, \
                      fast_customers, total_fast_customers,\
                      slow_customers, total_slow_customers,\
                     ds_customers, total_ds_customers,\
                     ps_customers, total_ps_customers)

In [ ]:
plot_price_impact('true_servetime', 'Service Time', all_customers, total_all_customers, \
                      fast_customers, total_fast_customers,\
                      slow_customers, total_slow_customers,\
                     ds_customers, total_ds_customers,\
                     ps_customers, total_ps_customers)

In [ ]:
plot_price_impact('charge_price', 'Monetary Cost', all_customers, total_all_customers, \
                      fast_customers, total_fast_customers,\
                      slow_customers, total_slow_customers,\
                     ds_customers, total_ds_customers,\
                     ps_customers, total_ps_customers)

In [ ]:
plot_price_impact('cost_from_wait', 'Delay Cost', all_customers, total_all_customers, \
                      fast_customers, total_fast_customers,\
                      slow_customers, total_slow_customers,\
                     ds_customers, total_ds_customers,\
                     ps_customers, total_ps_customers)

## Distribution Histograms

In [ ]:

#Plotting p_fast price vs. total cost 
    
def plot_total_cost_vs_price_all(gridsearch_folder, n_warmup_cust = 0):
    
    
    result_folders = [name for name in os.listdir(gridsearch_folder)]
    
    
    #total costs, averaged over all runs for the corresponding price
    avg_total_costs = []
    avg_cost_per_cust = []
    
    #total cost for all fast customers, averaged over all runs for the corresponding price
    avg_total_fast_costs = []
    avg_cost_per_fast_cust = []
    
    #total cost for all slow customers, averaged over all runs for the corresponding price
    avg_total_slow_costs = []
    avg_cost_per_slow_cust = []

    #total cost for all ds customers, averaged over all runs for the corresponding price
    avg_total_ds_costs = []
    avg_cost_per_ds_cust = []
    
    #total cost for all ps customers, averaged over all runs for the corresponding price
    avg_total_ps_costs = []
    avg_cost_per_ps_cust = []
    
    #corresponding p_fast price
    p_fasts = []
    
    
    for param_results in result_folders:
        
        with open(f'{gridsearch_folder}/{param_results}/params.json') as file:
            param_dict = json.load(file)
            
        num_runs = param_dict['num_runs']
        
        #total cost, will be averaged over all runs after for loop
        runs_total_cost = 0
        
        runs_fast_cost = 0
        
        runs_slow_cost = 0
        
        runs_ds_cost = 0
        
        runs_ps_cost = 0
        
        n_customers = param_dict['num_customers'] - n_warmup_cust
        
        
        for run in range(num_runs):
            run_file = f'{gridsearch_folder}/{param_results}/customer_results_run_{run}.csv'
            
            run_df = pd.read_csv(run_file ,index_col = 0, skiprows=range(1, n_warmup_cust+1))
            
            runs_total_cost += run_df['true_cost'].sum()
            runs_fast_cost += run_df[run_df['charger_choice']=='fast']['true_cost'].sum()
            runs_slow_cost += run_df[run_df['charger_choice']=='slow']['true_cost'].sum()
            runs_ds_cost += run_df[run_df['sens_type']=='delay']['true_cost'].sum()
            runs_ps_cost += run_df[run_df['sens_type']=='price']['true_cost'].sum()
            
        

        avg_total_costs.append(runs_total_cost/num_runs)
        avg_cost_per_cust.append(avg_total_costs[-1]/n_customers)

        avg_total_fast_costs.append(runs_fast_cost/num_runs)
        avg_cost_per_fast_cust.append(avg_total_fast_costs[-1]/n_customers)

        avg_total_slow_costs.append(runs_slow_cost/num_runs)
        avg_cost_per_slow_cust.append(avg_total_slow_costs[-1]/n_customers)

        avg_total_ds_costs.append(runs_ds_cost/num_runs)
        avg_cost_per_ds_cust.append(avg_total_ds_costs[-1]/n_customers)

        avg_total_ps_costs.append(runs_ps_cost/num_runs)
        avg_cost_per_ps_cust.append(avg_total_ps_costs[-1]/n_customers)

        p_fasts.append(param_dict['p_fast'])
        
    p_fasts, avg_total_costs, avg_cost_per_cust,\
    avg_total_fast_costs, avg_cost_per_fast_cust,\
    avg_total_slow_costs, avg_cost_per_slow_cust,\
    avg_total_ds_costs, avg_cost_per_ds_cust,\
    avg_total_ps_costs, avg_cost_per_ps_cust= zip(*sorted(zip(p_fasts, avg_total_costs, avg_cost_per_cust,\
                                                  avg_total_fast_costs, avg_cost_per_fast_cust,\
                                                  avg_total_slow_costs, avg_cost_per_slow_cust,\
                                                  avg_total_ds_costs, avg_cost_per_ds_cust,\
                                                  avg_total_ps_costs, avg_cost_per_ps_cust)))
                                                      
                                                      
    #Plot total cost 
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Line'))

    fig.update_layout(title='Effect of Fast Charger Price on Total Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')

    fig.show()


    #Plot per capita
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")
                                                      
    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='Line'))

    fig.update_layout(title='Effect of Fast Charger Price on Per Capita Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')

    fig.show()
                                                      
    #plot Total cost/total fast/total slow
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")
    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Total Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_fast_costs, mode='lines', name='Total Fast Charging Cost', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_slow_costs, mode='lines', name= 'Total Slow Charging Cost', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Total Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')

    fig.show()
                                                      
    #plot cost per capita from total/fast/slow 
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='Per Capita Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_fast_cust, mode='lines', name='Per Capita Cost (Fast Charging)', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_slow_cust, mode='lines', name= 'Per Capita Cost (Slow Charging)', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Per Capita Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')
                                                  
    fig.show()

                                                      
    #plot Total cost/total ds/total ps
                                              
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Total Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_ps_costs, mode='lines', name='Total Price-Sensitive Cost', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_ds_costs, mode='lines', name= 'Total Delay-Sensitive Cost', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Total Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')
        
    fig.show()
                                                      
                                                      
    #plot cost per customer from total/ds/ps
                                                      
                                                              
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")
    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='Per Capita Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_ps_cust, mode='lines', name='Per Capita Cost (Price-Sensitive)', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_ds_cust, mode='lines', name= 'Per Capita Cost (Delay-Sensitive)', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Per Capita Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')
    
    fig.show()
                                                                        
        
    minimizing_idx = np.argmin(avg_total_costs)
        
    #minimizing p_fast cost and the total cost associated with it 
    min_cost = avg_total_costs[minimizing_idx]
    minimizing_price = p_fasts[minimizing_idx]
        
    return  min_cost, minimizing_price
    


In [ ]:

#Plotting p_fast price vs. total cost 
    
def plot_total_cost_vs_price_all(gridsearch_folder, n_warmup_cust = 0):
    
    
    result_folders = [name for name in os.listdir(gridsearch_folder)]
    
    
    #total costs, averaged over all runs for the corresponding price
    avg_total_costs = []
    avg_cost_per_cust = []
    
    #total cost for all fast customers, averaged over all runs for the corresponding price
    avg_total_fast_costs = []
    avg_cost_per_fast_cust = []
    
    #total cost for all slow customers, averaged over all runs for the corresponding price
    avg_total_slow_costs = []
    avg_cost_per_slow_cust = []

    #total cost for all ds customers, averaged over all runs for the corresponding price
    avg_total_ds_costs = []
    avg_cost_per_ds_cust = []
    
    #total cost for all ps customers, averaged over all runs for the corresponding price
    avg_total_ps_costs = []
    avg_cost_per_ps_cust = []
    
    #corresponding p_fast price
    p_fasts = []
    
    
    for param_results in result_folders:
        
        with open(f'{gridsearch_folder}/{param_results}/params.json') as file:
            param_dict = json.load(file)
            
        num_runs = param_dict['num_runs']
        
        #total cost, will be averaged over all runs after for loop
        runs_total_cost = 0
        
        runs_fast_cost = 0
        
        runs_slow_cost = 0
        
        runs_ds_cost = 0
        
        runs_ps_cost = 0
        
        n_customers = param_dict['num_customers'] - n_warmup_cust
        
        
        for run in range(num_runs):
            run_file = f'{gridsearch_folder}/{param_results}/customer_results_run_{run}.csv'
            
            run_df = pd.read_csv(run_file ,index_col = 0, skiprows=range(1, n_warmup_cust+1))
            
            runs_total_cost += run_df['true_cost'].sum()
            runs_fast_cost += run_df[run_df['charger_choice']=='fast']['true_cost'].sum()
            runs_slow_cost += run_df[run_df['charger_choice']=='slow']['true_cost'].sum()
            runs_ds_cost += run_df[run_df['sens_type']=='delay']['true_cost'].sum()
            runs_ps_cost += run_df[run_df['sens_type']=='price']['true_cost'].sum()
            
        

        avg_total_costs.append(runs_total_cost/num_runs)
        avg_cost_per_cust.append(avg_total_costs[-1]/n_customers)

        avg_total_fast_costs.append(runs_fast_cost/num_runs)
        avg_cost_per_fast_cust.append(avg_total_fast_costs[-1]/n_customers)

        avg_total_slow_costs.append(runs_slow_cost/num_runs)
        avg_cost_per_slow_cust.append(avg_total_slow_costs[-1]/n_customers)

        avg_total_ds_costs.append(runs_ds_cost/num_runs)
        avg_cost_per_ds_cust.append(avg_total_ds_costs[-1]/n_customers)

        avg_total_ps_costs.append(runs_ps_cost/num_runs)
        avg_cost_per_ps_cust.append(avg_total_ps_costs[-1]/n_customers)

        p_fasts.append(param_dict['p_fast'])
        
    p_fasts, avg_total_costs, avg_cost_per_cust,\
    avg_total_fast_costs, avg_cost_per_fast_cust,\
    avg_total_slow_costs, avg_cost_per_slow_cust,\
    avg_total_ds_costs, avg_cost_per_ds_cust,\
    avg_total_ps_costs, avg_cost_per_ps_cust= zip(*sorted(zip(p_fasts, avg_total_costs, avg_cost_per_cust,\
                                                  avg_total_fast_costs, avg_cost_per_fast_cust,\
                                                  avg_total_slow_costs, avg_cost_per_slow_cust,\
                                                  avg_total_ds_costs, avg_cost_per_ds_cust,\
                                                  avg_total_ps_costs, avg_cost_per_ps_cust)))
                                                      
                                                      
    #Plot total cost 
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Line'))

    fig.update_layout(title='Effect of Fast Charger Price on Total Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')

    fig.show()


    #Plot per capita
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")
                                                      
    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='Line'))

    fig.update_layout(title='Effect of Fast Charger Price on Per Capita Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')

    fig.show()
                                                      
    #plot Total cost/total fast/total slow
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")
    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Total Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_fast_costs, mode='lines', name='Total Fast Charging Cost', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_slow_costs, mode='lines', name= 'Total Slow Charging Cost', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Total Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')

    fig.show()
                                                      
    #plot cost per capita from total/fast/slow 
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='Per Capita Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_fast_cust, mode='lines', name='Per Capita Cost (Fast Charging)', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_slow_cust, mode='lines', name= 'Per Capita Cost (Slow Charging)', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Per Capita Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')
                                                  
    fig.show()

                                                      
    #plot Total cost/total ds/total ps
                                              
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Total Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_ps_costs, mode='lines', name='Total Price-Sensitive Cost', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_ds_costs, mode='lines', name= 'Total Delay-Sensitive Cost', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Total Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')
        
    fig.show()
                                                      
                                                      
    #plot cost per customer from total/ds/ps
                                                      
                                                              
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")
    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='Per Capita Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_ps_cust, mode='lines', name='Per Capita Cost (Price-Sensitive)', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_ds_cust, mode='lines', name= 'Per Capita Cost (Delay-Sensitive)', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Per Capita Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')
    
    fig.show()
                                                                        
        
    minimizing_idx = np.argmin(avg_total_costs)
        
    #minimizing p_fast cost and the total cost associated with it 
    min_cost = avg_total_costs[minimizing_idx]
    minimizing_price = p_fasts[minimizing_idx]
        
    return  min_cost, minimizing_price
    


In [ ]:
#Fairness 

#histogram of all the same things plotted above, one by one

#What is the distribution of wait times/prices paid overall per customer? 
#or can just concat all customers from all runs + view it
#histograms, break out by customer type



#check to see we're capturing everything with the maxrange!!

def plot_customer_distributions(y_variable, title_str, n_warmup_customers, optimal_pfast, baseline_pfast,\
                                max_range_all, max_range_ds, max_range_ps):

    num_runs = None #should be the same number for both

    optimal_folder = None

    baseline_folder = None
    result_folders = [name for name in os.listdir(gridsearch_folder)]

    for folder in result_folders:

        with open(f'{gridsearch_folder}/{folder}/params.json') as file:
            param_dict = json.load(file)

        if param_dict["p_fast"]== baseline_pfast:
            baseline_folder = folder 
            num_runs = param_dict['num_runs']

        if param_dict["p_fast"]== optimal_pfast:
            optimal_folder = folder 
            num_runs = param_dict['num_runs']

    #Optimal price, distribution of costs over all people 

    cust_values = []

    cust_values_ds = []
    cust_values_ps = []

    for run in range(num_runs):
        run_file = f'{gridsearch_folder}/{optimal_folder}/customer_results_run_{run}.csv'
        runs_df=pd.read_csv(run_file, skiprows = range(1, n_warmup_customers+1), index_col = 0)
        cust_values += list(runs_df[y_variable])
        cust_values_ds += list(runs_df[runs_df['sens_type']=='delay'][y_variable])
        cust_values_ps += list(runs_df[runs_df['sens_type']=='price'][y_variable])

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values, marker_color='blue', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Optimal Price (All Customers)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_all]), 
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values_ds, marker_color='purple', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Optimal Price (Delay Sensitive)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_ds]), 
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values_ps, marker_color='orange', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Optimal Price (Price Sensitive)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_ps]),
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()


    #Baseline price 

    cust_values = []

    cust_values_ds = []
    cust_values_ps = []

    for run in range(num_runs):
        run_file = f'{gridsearch_folder}/{baseline_folder}/customer_results_run_{run}.csv'
        runs_df=pd.read_csv(run_file, skiprows = range(1, n_warmup_customers+1), index_col = 0)
        cust_values += list(runs_df[y_variable])
        cust_values_ds += list(runs_df[runs_df['sens_type']=='delay'][y_variable])
        cust_values_ps += list(runs_df[runs_df['sens_type']=='price'][y_variable])

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values, marker_color='blue', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Baseline Price (All Customers)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_all]), 
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values_ds, marker_color='purple', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Baseline Price (Delay Sensitive)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_ds]),
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values_ps, marker_color='orange', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Baseline Price (Price Sensitive)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_ps]), 
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()


                
    


In [ ]:
#Fairness - total cost

plot_customer_distributions('true_cost', 'Customer Cost', 1500, 2.15, 0,\
                                2000, 2000, 2000)

In [ ]:
#Fairness - total weight

plot_customer_distributions('true_total_wait', 'Customer Wait Times', 1500, 2.15, 0,\
                                800, 800, 800)

In [ ]:
#Fairness - total weight

plot_customer_distributions('true_servetime', 'Customer Service Times', 1500, 2.15, 0,\
                                800, 800, 800)

In [ ]:
#Fairness - total weight

plot_customer_distributions('true_queue_wait', 'Customer Queueing Times', 1500, 2.15, 0,\
                                250, 250, 250)

In [ ]:
#Fairness - total weight

plot_customer_distributions('charge_price', 'Customer Wait Times', 1500, 2.15, 0,\
                                200, , 800)

In [ ]:
#Fairness 

#histogram of all the same things plotted above, one by one

#What is the distribution of wait times/prices paid overall per customer? 
#or can just concat all customers from all runs + view it
#histograms, break out by customer type



#check to see we're capturing everything with the maxrange!!

def plot_customer_distributions(y_variable, title_str, n_warmup_customers, optimal_pfast, baseline_pfast,\
                                max_range_all, max_range_ds, max_range_ps):

    num_runs = None #should be the same number for both

    optimal_folder = None

    baseline_folder = None
    result_folders = [name for name in os.listdir(gridsearch_folder)]

    for folder in result_folders:

        with open(f'{gridsearch_folder}/{folder}/params.json') as file:
            param_dict = json.load(file)

        if param_dict["p_fast"]== baseline_pfast:
            baseline_folder = folder 
            num_runs = param_dict['num_runs']

        if param_dict["p_fast"]== optimal_pfast:
            optimal_folder = folder 
            num_runs = param_dict['num_runs']

    #Optimal price, distribution of costs over all people 

    cust_values = []

    cust_values_ds = []
    cust_values_ps = []

    for run in range(num_runs):
        run_file = f'{gridsearch_folder}/{optimal_folder}/customer_results_run_{run}.csv'
        runs_df=pd.read_csv(run_file, skiprows = range(1, n_warmup_customers+1), index_col = 0)
        cust_values += list(runs_df[y_variable])
        cust_values_ds += list(runs_df[runs_df['sens_type']=='delay'][y_variable])
        cust_values_ps += list(runs_df[runs_df['sens_type']=='price'][y_variable])

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values, marker_color='blue', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Optimal Price (All Customers)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_all]), 
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values_ds, marker_color='purple', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Optimal Price (Delay Sensitive)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_ds]), 
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values_ps, marker_color='orange', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Optimal Price (Price Sensitive)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_ps]),
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()


    #Baseline price 

    cust_values = []

    cust_values_ds = []
    cust_values_ps = []

    for run in range(num_runs):
        run_file = f'{gridsearch_folder}/{baseline_folder}/customer_results_run_{run}.csv'
        runs_df=pd.read_csv(run_file, skiprows = range(1, n_warmup_customers+1), index_col = 0)
        cust_values += list(runs_df[y_variable])
        cust_values_ds += list(runs_df[runs_df['sens_type']=='delay'][y_variable])
        cust_values_ps += list(runs_df[runs_df['sens_type']=='price'][y_variable])

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values, marker_color='blue', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Baseline Price (All Customers)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_all]), 
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values_ds, marker_color='purple', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Baseline Price (Delay Sensitive)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_ds]),
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()

    # Create a histogram with red color and normalized between [0, 1]
    fig = go.Figure(data=[go.Histogram(x=cust_values_ps, marker_color='orange', histnorm='probability')])

    # Update layout
    fig.update_layout(
        title=f"Distribution of {title_str} Under the Baseline Price (Price Sensitive)",
        xaxis_title=f"{title_str}",
        xaxis=dict(range=[0, max_range_ps]), 
        yaxis_title="Empirical Probability"
    )

    # Show the plot
    fig.show()


                
    


In [ ]:
#Fairness - total weight

plot_customer_distributions('true_queue_wait', 'Customer Wait Times', 1500, 2.15, 0,\
                                800, 800, 800)

In [ ]:
max_range_all =1000
max_range_ds = 1000
max_range_ps = 1000
y_variable = 'true_total_wait'
title_str = 'Customer Time in System'
n_warmup_customers = 1500
optimal_pfast = 2.15 
baseline_pfast = 0

plot_total_cost_vs_price_all(gridsearch_folder, n_warmup_cust = 0)

In [ ]:
#may want to also consider plotting for each type of ds/ps customer how their total cost breaks down 
#in terms of monetary vs delay costs

In [ ]:

#Plotting p_fast price vs. total cost 
    
def plot_total_cost_vs_price_all(gridsearch_folder, n_warmup_cust = 0):
    
    
    result_folders = [name for name in os.listdir(gridsearch_folder)]
    
    
    #total costs, averaged over all runs for the corresponding price
    avg_total_costs = []
    avg_cost_per_cust = []
    
    #total cost for all fast customers, averaged over all runs for the corresponding price
    avg_total_fast_costs = []
    avg_cost_per_fast_cust = []
    
    #total cost for all slow customers, averaged over all runs for the corresponding price
    avg_total_slow_costs = []
    avg_cost_per_slow_cust = []

    #total cost for all ds customers, averaged over all runs for the corresponding price
    avg_total_ds_costs = []
    avg_cost_per_ds_cust = []
    
    #total cost for all ps customers, averaged over all runs for the corresponding price
    avg_total_ps_costs = []
    avg_cost_per_ps_cust = []
    
    #corresponding p_fast price
    p_fasts = []
    
    
    for param_results in result_folders:
        
        with open(f'{gridsearch_folder}/{param_results}/params.json') as file:
            param_dict = json.load(file)
            
        num_runs = param_dict['num_runs']
        
        #total cost, will be averaged over all runs after for loop
        runs_total_cost = 0
        
        runs_fast_cost = 0
        
        runs_slow_cost = 0
        
        runs_ds_cost = 0
        
        runs_ps_cost = 0
        
        n_customers = param_dict['num_customers'] - n_warmup_cust
        
        
        for run in range(num_runs):
            run_file = f'{gridsearch_folder}/{param_results}/customer_results_run_{run}.csv'
            
            run_df = pd.read_csv(run_file ,index_col = 0, skiprows=range(1, n_warmup_cust+1))
            
            runs_total_cost += run_df['true_cost'].sum()
            runs_fast_cost += run_df[run_df['charger_choice']=='fast']['true_cost'].sum()
            runs_slow_cost += run_df[run_df['charger_choice']=='slow']['true_cost'].sum()
            runs_ds_cost += run_df[run_df['sens_type']=='delay']['true_cost'].sum()
            runs_ps_cost += run_df[run_df['sens_type']=='price']['true_cost'].sum()
            
        

        avg_total_costs.append(runs_total_cost/num_runs)
        avg_cost_per_cust.append(avg_total_costs[-1]/n_customers)

        avg_total_fast_costs.append(runs_fast_cost/num_runs)
        avg_cost_per_fast_cust.append(avg_total_fast_costs[-1]/n_customers)

        avg_total_slow_costs.append(runs_slow_cost/num_runs)
        avg_cost_per_slow_cust.append(avg_total_slow_costs[-1]/n_customers)

        avg_total_ds_costs.append(runs_ds_cost/num_runs)
        avg_cost_per_ds_cust.append(avg_total_ds_costs[-1]/n_customers)

        avg_total_ps_costs.append(runs_ps_cost/num_runs)
        avg_cost_per_ps_cust.append(avg_total_ps_costs[-1]/n_customers)

        p_fasts.append(param_dict['p_fast'])
        
    p_fasts, avg_total_costs, avg_cost_per_cust,\
    avg_total_fast_costs, avg_cost_per_fast_cust,\
    avg_total_slow_costs, avg_cost_per_slow_cust,\
    avg_total_ds_costs, avg_cost_per_ds_cust,\
    avg_total_ps_costs, avg_cost_per_ps_cust= zip(*sorted(zip(p_fasts, avg_total_costs, avg_cost_per_cust,\
                                                  avg_total_fast_costs, avg_cost_per_fast_cust,\
                                                  avg_total_slow_costs, avg_cost_per_slow_cust,\
                                                  avg_total_ds_costs, avg_cost_per_ds_cust,\
                                                  avg_total_ps_costs, avg_cost_per_ps_cust)))
                                                      
                                                      
    #Plot total cost 
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Line'))

    fig.update_layout(title='Effect of Fast Charger Price on Total Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')

    fig.show()


    #Plot per capita
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")
                                                      
    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='Line'))

    fig.update_layout(title='Effect of Fast Charger Price on Per Capita Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')

    fig.show()
                                                      
    #plot Total cost/total fast/total slow
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")
    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Total Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_fast_costs, mode='lines', name='Total Fast Charging Cost', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_slow_costs, mode='lines', name= 'Total Slow Charging Cost', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Total Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')

    fig.show()
                                                      
    #plot cost per capita from total/fast/slow 
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='Per Capita Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_fast_cust, mode='lines', name='Per Capita Cost (Fast Charging)', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_slow_cust, mode='lines', name= 'Per Capita Cost (Slow Charging)', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Per Capita Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')
                                                  
    fig.show()

                                                      
    #plot Total cost/total ds/total ps
                                              
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Total Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_ps_costs, mode='lines', name='Total Price-Sensitive Cost', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_ds_costs, mode='lines', name= 'Total Delay-Sensitive Cost', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Total Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')
        
    fig.show()
                                                      
                                                      
    #plot cost per customer from total/ds/ps
                                                      
                                                              
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")
    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='Per Capita Cost', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_ps_cust, mode='lines', name='Per Capita Cost (Price-Sensitive)', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_ds_cust, mode='lines', name= 'Per Capita Cost (Delay-Sensitive)', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Per Capita Cost',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Cost (Delay and Price)')
    
    fig.show()
                                                                        
        
    minimizing_idx = np.argmin(avg_total_costs)
        
    #minimizing p_fast cost and the total cost associated with it 
    min_cost = avg_total_costs[minimizing_idx]
    minimizing_price = p_fasts[minimizing_idx]
        
    return  min_cost, minimizing_price
    


In [ ]:
plot_total_cost_vs_price_all(gridsearch_folder, 1500)

In [ ]:

#Plotting p_fast price vs. wait time in the queue
    
def plot_total_wait_vs_price_all(gridsearch_folder, n_warmup_cust = 0):
    
    """
    Plot how each group experiences wait time in the queue, ignoring time during service because obviously 
    the people who go through the slow vs. fast chargers have different service times
    
    In the variables below, "cost" is actually wait time (didn't want to rename all the variables from before)
    """
    
    
    result_folders = [name for name in os.listdir(gridsearch_folder)]
    
    
    #total costs, averaged over all runs for the corresponding price
    avg_total_costs = []
    avg_cost_per_cust = []
    
    #total cost for all fast customers, averaged over all runs for the corresponding price
    avg_total_fast_costs = []
    avg_cost_per_fast_cust = []
    
    #total cost for all slow customers, averaged over all runs for the corresponding price
    avg_total_slow_costs = []
    avg_cost_per_slow_cust = []

    #total cost for all ds customers, averaged over all runs for the corresponding price
    avg_total_ds_costs = []
    avg_cost_per_ds_cust = []
    
    #total cost for all ps customers, averaged over all runs for the corresponding price
    avg_total_ps_costs = []
    avg_cost_per_ps_cust = []
    
    #corresponding p_fast price
    p_fasts = []
    
    
    for param_results in result_folders:
        
        with open(f'{gridsearch_folder}/{param_results}/params.json') as file:
            param_dict = json.load(file)
            
        num_runs = param_dict['num_runs']
        
        #total cost, will be averaged over all runs after for loop
        runs_total_cost = 0
        
        runs_fast_cost = 0
        
        runs_slow_cost = 0
        
        runs_ds_cost = 0
        
        runs_ps_cost = 0
        
        num_total_customers = param_dict['num_customers'] - n_warmup_cust
        
        
        for run in range(num_runs):
            run_file = f'{gridsearch_folder}/{param_results}/customer_results_run_{run}.csv'
            
            run_df = pd.read_csv(run_file ,index_col = 0, skiprows=range(1, n_warmup_cust+1))
            
            runs_total_cost += run_df['true_queue_wait'].sum()
            runs_fast_cost += run_df[run_df['charger_choice']=='fast']['true_queue_wait'].sum()
            runs_slow_cost += run_df[run_df['charger_choice']=='slow']['true_queue_wait'].sum()
            runs_ds_cost += run_df[run_df['sens_type']=='delay']['true_queue_wait'].sum()
            runs_ps_cost += run_df[run_df['sens_type']=='price']['true_queue_wait'].sum()
            
        

        avg_total_costs.append(runs_total_cost/num_runs)
        avg_cost_per_cust.append(avg_total_costs[-1]/n_customers)

        avg_total_fast_costs.append(runs_fast_cost/num_runs)
        avg_cost_per_fast_cust.append(avg_total_fast_costs[-1]/n_customers)

        avg_total_slow_costs.append(runs_slow_cost/num_runs)
        avg_cost_per_slow_cust.append(avg_total_slow_costs[-1]/n_customers)

        avg_total_ds_costs.append(runs_ds_cost/num_runs)
        avg_cost_per_ds_cust.append(avg_total_ds_costs[-1]/n_customers)

        avg_total_ps_costs.append(runs_ps_cost/num_runs)
        avg_cost_per_ps_cust.append(avg_total_ps_costs[-1]/n_customers)

        p_fasts.append(param_dict['p_fast'])
        
    p_fasts, avg_total_costs, avg_cost_per_cust,\
    avg_total_fast_costs, avg_cost_per_fast_cust,\
    avg_total_slow_costs, avg_cost_per_slow_cust,\
    avg_total_ds_costs, avg_cost_per_ds_cust,\
    avg_total_ps_costs, avg_cost_per_ps_cust= zip(*sorted(zip(p_fasts, avg_total_costs, avg_cost_per_cust,\
                                                  avg_total_fast_costs, avg_cost_per_fast_cust,\
                                                  avg_total_slow_costs, avg_cost_per_slow_cust,\
                                                  avg_total_ds_costs, avg_cost_per_ds_cust,\
                                                  avg_total_ps_costs, avg_cost_per_ps_cust)))
                                                      
                                                      
    #Plot total cost 
    
    print(p_fasts)
                                                      
    fig = go.Figure()
    
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Line'))

    fig.update_layout(title='Effect of Fast Charger Price on Total Wait Time (Queue)',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Time in Queue (Minutes)')

    fig.show()


    #Plot per capita
    fig = go.Figure()
    
    fig.update_yaxes(rangemode="tozero")
                                                      
    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='Line'))

    fig.update_layout(title='Effect of Fast Charger Price on Wait Time Per Customer (Queue)',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Time in Queue (Minutes')

    fig.show()
                                                      
    #plot Total cost/total fast/total slow
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='Total Wait Time', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_fast_costs, mode='lines', name='Total Fast Charging Wait Time', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_slow_costs, mode='lines', name= 'Total Slow Charging Wait Time', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Total Wait Time (Queue)',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Time in Queue (Minutes')

    fig.show()
                                                      
    #plot cost per capita from total/fast/slow 
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='All', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_fast_cust, mode='lines', name='Fast Charging', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_slow_cust, mode='lines', name= 'Slow Charging', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Wait Time Per Customer (Queue)',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Time in Queue (Minutes)')
                                                  
    fig.show()

                                                      
    #plot Total cost/total ds/total ps
                                              
                                                      
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_costs, mode='lines', name='All', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_ps_costs, mode='lines', name='Price-Sensitive', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_total_ds_costs, mode='lines', name= 'Delay-Sensitive', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Total Wait',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Time in Queue (Minutes)')
        
    fig.show()
                                                      
                                                      
    #plot cost per customer from total/ds/ps
                                                      
                                                              
    fig = go.Figure()
    fig.update_yaxes(rangemode="tozero")

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_cust, mode='lines', name='All', line=dict(color='blue')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_ps_cust, mode='lines', name='Price-Sensitive', line=dict(color='green')))

    fig.add_trace(go.Scatter(x=p_fasts, y=avg_cost_per_ds_cust, mode='lines', name= 'Delay-Sensitive', line=dict(color='red')))

    fig.update_layout(title='Effect of Fast Charger Price on Per Capita Wait',
                          xaxis_title='Fast Charger Price',
                          yaxis_title='Time in Queue (Minutes)')
    
    fig.show()
                                                                        
        
    minimizing_idx = np.argmin(avg_cost_per_cust)
        
    #wait-time minimizing p_fast cost and the per-customer wait associated with it
    min_cost = avg_cost_per_cust[minimizing_idx]
    minimizing_price = p_fasts[minimizing_idx]
        
    return  min_cost, minimizing_price
    


In [ ]:
plot_total_wait_vs_price_all(gridsearch_folder)